OBJETIVO: prever se o cliente do banco irá fazer um depósito a prazo.

Algoritmos usados:
<ul style='list-style-type: None'>
<li>Random Forest</li>
<li>XGBoost</li>
<li>Redes Neurais</li>
</ul>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
import numpy as np
from sklearn.preprocessing import LabelEncoder


c:\Users\marce\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (2.0.5) or chardet (4.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
df = pd.read_csv('bank-full.csv', sep=';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
# Alterar o nome da coluna-alvo (y) para 'subscribed'
df = df.rename(columns={'y': 'subscribed'})
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,subscribed
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   age         45211 non-null  int64 
 1   job         45211 non-null  object
 2   marital     45211 non-null  object
 3   education   45211 non-null  object
 4   default     45211 non-null  object
 5   balance     45211 non-null  int64 
 6   housing     45211 non-null  object
 7   loan        45211 non-null  object
 8   contact     45211 non-null  object
 9   day         45211 non-null  int64 
 10  month       45211 non-null  object
 11  duration    45211 non-null  int64 
 12  campaign    45211 non-null  int64 
 13  pdays       45211 non-null  int64 
 14  previous    45211 non-null  int64 
 15  poutcome    45211 non-null  object
 16  subscribed  45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [6]:
# Há valores faltantes (NaN)?
df.isna().sum()

age           0
job           0
marital       0
education     0
default       0
balance       0
housing       0
loan          0
contact       0
day           0
month         0
duration      0
campaign      0
pdays         0
previous      0
poutcome      0
subscribed    0
dtype: int64

Transformar as colunas categóricas que serão usadas (job, education, balance, loan, campaing) no modelo para colunas numéricas

In [7]:
label_encoder = LabelEncoder()

In [8]:
df.iloc[:, 1] = label_encoder.fit_transform(df.iloc[:, 1])
df.iloc[:, 3] = label_encoder.fit_transform(df.iloc[:, 3])
df.iloc[:, 16] = label_encoder.fit_transform(df.iloc[:, 16])

In [10]:
# Transformar a coluna 'loan' (assumindo que é sim/não) em 0s e 1s
df['loan'] = df['loan'].apply(lambda x: 1 if x == 'yes' else 0)

Definir as variáveis X e y

In [11]:
X = df[['job', 'education', 'balance', 'loan', 'campaign']].values 
y = df['subscribed'].values 

Separando em dados de treino e teste

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)
y_train = np.array(y_train).astype(int)
y_test = np.array(y_test).astype(int)

# Aplicar o LabelEncoder aos rótulos de treinamento
#y_train_encoded = label_encoder.fit_transform(y_train)

# Aplicar o LabelEncoder aos rótulos de teste
#y_test_encoded = label_encoder.transform(y_test)

RANDOM FOREST

Criar e treinar o modelo

In [50]:
rnd_forest = RandomForestClassifier(n_estimators=100, random_state=5, class_weight='balanced')
rnd_forest.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=5)

Previsões com dados de teste

In [51]:
rnd_forest_pred = rnd_forest.predict(X_test)

Avaliação do modelo

In [52]:
acuracia = accuracy_score(y_test, rnd_forest_pred)
print(f'Acurácia:\n {acuracia * 100:.2f}%')

Acurácia:
 82.33%


In [53]:
confusao = confusion_matrix(y_test, rnd_forest_pred)
confusao

array([[7148,  849],
       [ 749,  297]], dtype=int64)

Apesar de a acurácia estar bem boa, a matriz de confusão mostra valors altos de falsos positivos (849) e falsos negativos (749).

XGBOOST

Criar modelo

In [84]:
xgb = XGBClassifier(max_depth=15, learning_rate=1, alpha=0.1)
xgb.fit(X_train, y_train)

XGBClassifier(alpha=0.1, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

Previsões com dados de teste

In [85]:
xgb_pred = xgb.predict(X_test)

Avaliação

In [86]:
acuracia = accuracy_score(y_test, xgb_pred)
print(f'Acurácia:\n {acuracia * 100:.2f}%')
print(f'Matriz de Confusão:\n {confusion_matrix(y_test, xgb_pred)}')

Acurácia:
 85.30%
Matriz de Confusão:
 [[7596  401]
 [ 928  118]]


A Acurácia está boa mas há 928 falsos positivos, o que não é bom. Vamos tentar balancear a classe

In [88]:
from imblearn.over_sampling import SMOTE

In [89]:
# Aplicar SMOTE para balanceamento de classes
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

In [90]:
xgb = XGBClassifier(max_depth=15, learning_rate=1, alpha=0.1)
xgb.fit(X_train, y_train)

XGBClassifier(alpha=0.1, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [91]:
acuracia = accuracy_score(y_test, xgb_pred)
print(f'Acurácia:\n {acuracia * 100:.2f}%')
print(f'Matriz de Confusão:\n {confusion_matrix(y_test, xgb_pred)}')

Acurácia:
 85.30%
Matriz de Confusão:
 [[7596  401]
 [ 928  118]]


Nada mudou, vamos ver como fica com Redes Neurais

REDES NEURAIS

In [92]:
from sklearn.preprocessing import StandardScaler

In [93]:
scaler = StandardScaler()

Normalizar as características de entrada

In [94]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

Construir o modelo de Redes Neurais

In [95]:
rn_model = Sequential()

Adicionar camadas à rede neural

In [96]:
rn_model.add(Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]))
rn_model.add(Dense(units=64, activation='relu'))
rn_model.add(Dense(units=1, activation='sigmoid'))

Compilar o modelo

In [97]:
rn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Treinamento do modelo

In [98]:
rn_model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test))

Epoch 1/10
1131/1131 [==============================] - 2s 1ms/step - loss: 0.3578 - accuracy: 0.8811 - val_loss: 0.3440 - val_accuracy: 0.8843
Epoch 2/10
1131/1131 [==============================] - 1s 1ms/step - loss: 0.3507 - accuracy: 0.8827 - val_loss: 0.3434 - val_accuracy: 0.8843
Epoch 3/10
1131/1131 [==============================] - 1s 908us/step - loss: 0.3496 - accuracy: 0.8827 - val_loss: 0.3444 - val_accuracy: 0.8843
Epoch 4/10
1131/1131 [==============================] - 1s 984us/step - loss: 0.3491 - accuracy: 0.8827 - val_loss: 0.3432 - val_accuracy: 0.8843
Epoch 5/10
1131/1131 [==============================] - 1s 939us/step - loss: 0.3490 - accuracy: 0.8827 - val_loss: 0.3437 - val_accuracy: 0.8843
Epoch 6/10
1131/1131 [==============================] - 1s 916us/step - loss: 0.3488 - accuracy: 0.8827 - val_loss: 0.3425 - val_accuracy: 0.8843
Epoch 7/10
1131/1131 [==============================] - 1s 899us/step - loss: 0.3481 - accuracy: 0.8827 - val_loss: 0.3438 - val

Avaliação do modelo

In [99]:
loss, accuracy = rn_model.evaluate(X_test_scaled, y_test)
print(f'Acurácia: {accuracy * 100:.2f}%')

283/283 [==============================] - 0s 722us/step - loss: 0.3440 - accuracy: 0.8843
Acurácia: 88.43%


RESULTADO:

A rede neural foi treinada com sucesso e alcançou uma acurácia de 88.43% nos dados de teste. Isso significa que o modelo é capaz de classificar corretamente aproximadamente 88.43% das amostras no conjunto de teste.

Agora, podemos usar esse modelo para fazer previsões em novos dados ou continuar a otimizá-lo se desejar melhorar ainda mais o desempenho

PREVISÕES COM NOVOS DADOS

Para fins de demonstração, vou criar uma pequena simulação de dados de teste. Isso pode ser feito criando um pequeno conjunto de dados com as mesmas características que você usou durante o treinamento:

In [121]:
# Simulação de novos dados (apenas para demonstração)
novos_dados = [['management', 'tertiary', 2143, 'no', 1]]

In [122]:
# Inicializar LabelEncoders para codificar variáveis categóricas
job_encoder = LabelEncoder()
education_encoder = LabelEncoder()
loan_encoder = LabelEncoder()

In [123]:
# Aplicar Label Encoding às variáveis categóricas
novos_dados[0][0] = job_encoder.fit_transform([novos_dados[0][0]])[0]
novos_dados[0][1] = education_encoder.fit_transform([novos_dados[0][1]])[0]
novos_dados[0][3] = loan_encoder.fit_transform([novos_dados[0][3]])[0]

In [124]:
# Converter 'balance' e 'campaign' para números se já não estiverem
novos_dados[0][2] = int(novos_dados[0][2])
novos_dados[0][4] = int(novos_dados[0][4])

In [125]:
novos_dados_array = np.array(novos_dados).astype(float)

In [127]:
# Faz previsões usando o modelo treinado e os dados simulados
previsoes = rn_model.predict(novos_dados_array)

In [128]:
# Converte probabilidades em classes usando um threshold (por exemplo, 0.5)
threshold = 0.5
classes_previstas_simuladas = (previsoes > threshold).astype(int)

# Imprime as classes previstas
print("Classes Previstas para os Dados Simulados:", classes_previstas_simuladas)

Classes Previstas para os Dados Simulados: [[0]]


Com base nos novos dados de entrada, o modelo de rede neural previu a classe 0, ou seja, o cliente não vai realizar o depósito.